In [1]:
context = """The Great Wall stretches from Dandong in the east to Lop Lake in the west, along an arc that roughly delineates the southern edge of Inner Mongolia. A comprehensive archaeological survey, using advanced technologies, has concluded that the Ming walls measure 8,850 km (5,500 mi).[4] This is made up of 6,259 km (3,889 mi) sections of actual wall, 359 km (223 mi) of trenches and 2,232 km (1,387 mi) of natural defensive barriers such as hills and rivers.[4] Another archaeological survey found that the entire wall with all of its branches measure out to be 21,196 km (13,171 mi).[5]"""
query = "Ming walls long"
answer = ""

In [2]:
print (context)

The Great Wall stretches from Dandong in the east to Lop Lake in the west, along an arc that roughly delineates the southern edge of Inner Mongolia. A comprehensive archaeological survey, using advanced technologies, has concluded that the Ming walls measure 8,850 km (5,500 mi).[4] This is made up of 6,259 km (3,889 mi) sections of actual wall, 359 km (223 mi) of trenches and 2,232 km (1,387 mi) of natural defensive barriers such as hills and rivers.[4] Another archaeological survey found that the entire wall with all of its branches measure out to be 21,196 km (13,171 mi).[5]


In [3]:
# trans to data json
import pandas as pd
online_data = pd.DataFrame({"Query":query, "Context":context, "phrase":answer}, columns=["Query", "Context", "phrase"], index=[0])

import hashlib
def GetHashCode(context):
    hash = hashlib.md5()
    hash.update(context.encode('utf-8'))
    return hash.hexdigest()

import json
import os

# 只处理lable为1的row
target_dev_data = {}
target_dev_data['version'] = '1.1'
target_dev_data['data'] = list()

item = {}
item['paragraphs'] = list()
item['title'] = 'Online'

# 处理一个context对应一个quesiton，对应多个answer的情况
paragraphs_dict = {}

num = 0

def transEachRow(row):
        
    # 新生成的context
    paragraph = {}
    paragraph['qas'] = list()
    paragraph['context'] = row['Context'].strip()

    phrase = {}
    phrase['id'] = GetHashCode(row['Context'] + ' ' + row['Query'])
    phrase['question'] = row['Query'].strip()
    phrase['answers'] = list()

    phrase_answer = {}

    row['phrase'] = row['phrase'].strip()  # phrase 可能含有空格
    phrase_answer['text'] = row['phrase']

    # 过滤长度大于32的phrase
    global num
    if len(row['phrase']) >= 32:
        num += 1
        return 

    phrase_answer['answer_start'] = paragraph['context'].find(row['phrase'])
    if phrase_answer['answer_start'] == -1:
        print (paragraph['context'])
        print (row['phrase'])
        print (row)
        print (row.values)
        print 
    phrase['answers'].append(phrase_answer)

    paragraph['qas'].append(phrase)

    paragraph_key = GetHashCode(paragraph['context'])
    # 判断是否存在该context， 一个context只有一个quesiotn
    if paragraph_key not in paragraphs_dict:
        paragraphs_dict[paragraph_key] = paragraph
    else:
        paragraphs_dict[paragraph_key]['qas'][0]['answers'].append(phrase_answer)

online_data.apply(transEachRow, axis=1)
item['paragraphs'] = list(paragraphs_dict.values())

# print (len(list(paragraphs_dict.values())))
target_dev_data['data'].append(item)

# print (target_dev_data)
target_dev_path = '/home/t-honli/data/online/online.json'
json.dump(target_dev_data, open(target_dev_path, 'w'))

# print ('phrase length >= 32: %d' % (num))

In [4]:
# prepro json to test data
import os
prepro_file = 'squad.prepro'
source_dir = '/home/t-honli/data/online'
target_dir = '/home/t-honli/bi-att-flow/data/online'
file_name = 'online.json'

# change workspace
work_dir = '/home/t-honli/bi-att-flow'
os.chdir(work_dir)

os.system("python -m {} --online=True --data_file={} --source_dir={} --target_dir={}".format(prepro_file, file_name, source_dir, target_dir))

256

In [5]:
# test data  python -m basic.cli --len_opt --cluster --data_dir=data/online --online=True
data_dir = 'data/online'
ans_num = 2
os.system("python -m basic.cli --len_opt --cluster --data_dir={} --online=True --topk=2".format(data_dir))

256

In [6]:
# show answer
import json
answer_dir = '/home/t-honli/bi-att-flow/out/EQnA/03-07-2017/answer'
name = 'online-020000.json'
answer = json.load(open(os.path.join(answer_dir, name), "r"))
for key,val in list(answer.items()):
    if key != 'scores':
        phrases = val.split('|')
        scores = answer['scores'][key].split('|')
        for phrase, score in zip(phrases, scores):
            print (phrase, score)

8,850 km 0.0238
6,259 km 0.0173


In [10]:
import time
ISOTIMEFORMAT='%Y-%m-%d_%X'
print (time.strftime( ISOTIMEFORMAT, time.localtime( time.time() )))

2017-07-07_12:43:01
